## Build SN CDFs and calculate KS and Anderson-Darling tests.

Extra paper plots are created at the end.

In [ ]:
import numpy as np
import pandas as pd
import math
import os.path
sys.path.append(os.path.join(os.getenv('HOME'),'workspace','galbase'))
from gal_data import gal_data
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astropy.utils.console import ProgressBar
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.cm as cm
from operator import itemgetter
from astropy.coordinates import SkyCoord
import astropy.units as u
from reproject import reproject_interp
from scipy.stats import ks_2samp

### Build the SN CDFs

In [ ]:
sn_conv2p0 = pd.read_csv('../sne_project/sample_processing/2kpc.csv')

In [ ]:
samp = sn_conv2p0
# narrow down the parent table by SN type
bitmask = (((samp.TYPE == 'Ia') |
            (samp.TYPE == 'II') |
            (samp.TYPE == 'II P') |
            (samp.TYPE == 'IIL') |
            (samp.TYPE == 'IIb') |
            (samp.TYPE == 'Ib') |
            (samp.TYPE == 'Ic') |
            (samp.TYPE == 'Ibc') |
            (samp.TYPE == 'Ib/c')
           ))
samp_type = samp[bitmask]
print(len(samp_type))

Calculate and add the fiducial models (see "Extra Plots for Paper" below) to the SN CDF plots.

In [ ]:
def cdf_model(l_fid):
    dr = 0.01
    rgal = np.arange(0., 2.0, 0.001)
    fid_disk = np.exp(-1.0*rgal/l_fid)
    annulus_area = 2 * np.pi * rgal * dr
    total_flux_in_annulus = fid_disk * annulus_area
    cdf = np.cumsum(total_flux_in_annulus)
    norm_cdf = cdf / cdf[-1]
    return(fid_disk,norm_cdf)

In [ ]:
# calculate for our galaxy with l_fid = 0.25
fid_disk_stan, cdf_gal = cdf_model(l_fid=0.25)

# calculate for the SN distributions
fid_disk_con, cdf_con = cdf_model(l_fid=0.125) # concentrated
fid_disk_ext, cdf_ext = cdf_model(l_fid=0.5)    # extended

In [ ]:
standard = []
standard_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_stan)/np.sum(2*np.pi*rgal*fid_disk_stan))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    standard.append(random_cdf)
    standard_rad.append(random_rad)

concentrated = []
concentrated_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_con)/np.sum(2*np.pi*rgal*fid_disk_con))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    concentrated.append(random_cdf)
    concentrated_rad.append(random_rad)
    
extended = []
extended_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_ext)/np.sum(2*np.pi*rgal*fid_disk_ext))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    extended.append(random_cdf)
    extended_rad.append(random_rad)

In [ ]:
standard = np.array(standard)
standard = standard[np.isfinite(standard)]
standard = np.sort(standard)

extended = np.array(extended)
extended = extended[np.isfinite(extended)]
extended = np.sort(extended)

concentrated = np.array(concentrated)
concentrated = concentrated[np.isfinite(concentrated)]
concentrated = np.sort(concentrated)

Plot the CDFs for all the WISE and GALEX bands.

In [ ]:
def plot_cdfs(samp, res):
    # stuff for the plots
    font = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 18,
            }

    font1 = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'light',
            'style': 'italic',
            'size': 14,
            }
    font_axes = {'family': 'sans-serif',
                'color':  'black',
                'weight': 'normal',
                'size': 16,
                }
    
    legend  = ['Ia', 'II', 'Ib/c']
    colors = ['tab:blue', 'tab:red', 'tab:orange']
    fig, axes = plt.subplots(3, 2, figsize=(12,12))
    axes = axes.flatten()
    
    sn_type  = ['Ia', 'II',  'Ib']
    alias1   = ['Ia', 'II P','Ic']
    alias2   = ['Ia', 'IIL', 'Ib/c']
    alias3   = ['Ia', 'II',  'Ibc']
    alias4   = ['Ia', 'II',  'IIb']
    bands    = ['W1', 'W2', 'W3', 'W4', 'NUV', 'FUV']
    wavelengths = ['3.4 $\mu$m','4.5 $\mu$m','12 $\mu$m','22 $\mu$m','231 nm','154 nm']
    
    for i in range(len(bands)):
        # plot the toy models
        axes[i].plot(np.array(extended), np.linspace(0,1.0,len(extended)), color='#666666', ls='--', alpha=0.5)
        axes[i].plot(np.array(concentrated), np.linspace(0,1.0,len(concentrated)), color='#666666', ls='--',alpha=0.5)
            

    # for KS test
    Ia  = []
    II  = []
    Ibc = []
    types = [Ia, II, Ibc]

    # loop over each SN type
    gals = [[],[],[],[],[],[]]
    for j in range(len(sn_type)):

        # narrow down the parent table by SN type
        bitmask = (((samp.TYPE == sn_type[j]) |
                    (samp.TYPE == alias1[j]) |
                    (samp.TYPE == alias2[j]) |
                    (samp.TYPE == alias3[j]) |
                    (samp.TYPE == alias4[j])))
        samp_type = samp[bitmask]

        # loop over each SN of a particular type
        cdf_vals = [[],[],[],[],[],[]]
        sn_rads = []
        bar = ProgressBar(len(samp_type), ipython_widget=True)
        for index, row in samp_type.iterrows():
            pgc = row['PGC']
            sn  = row['NAME']

            # grab the galaxy cdfs
            for i in range(len(bands)):
                try:
                    gal_cdf_file = pd.read_csv('../sne_project/galaxy_control_tables/gal_cdfs_%s/%s_%s_%s.csv' % (res, pgc, bands[i], res))
                except FileNotFoundError:
                    continue
                
                # grab the columns of the galaxy cdfs
                
                rad = np.array(round(gal_cdf_file['RADIUS'],5)) # round so np.interp works
                cdf = np.array(gal_cdf_file['CDF'])

                try:
                    sn_rad = round(row['SN_RAD'],5)
                    sn_rads += [sn_rad]

                    # get SN cdf val     
                    sn_val_cdf = np.interp(sn_rad, rad, cdf)
                    gals[i].append(pgc)
                except:
                    sn_val_cdf = np.nan

                # SN cdf vals
                cdf_vals[i].append(sn_val_cdf)
                
            bar.update()
        # make the plots per band
        for i in range(len(bands)):
            
            # plot the SN distributions
            cdf_vals[i] = np.array(cdf_vals[i])
            cdf_vals[i] = cdf_vals[i][np.isfinite(cdf_vals[i])]
            cdf_vals[i] = np.sort(cdf_vals[i])
            types[j].append(cdf_vals[i]) # FOR KS TEST
            axes[i].plot(cdf_vals[i], np.linspace(0, 1.0, len(cdf_vals[i])), color=colors[j], label=legend[j] + ' (%s SNe)' % (len(cdf_vals[i])), linewidth=2)
   
    # make the plots prettier 
    for i in range(len(bands)):
        axes[i].set_xlabel('Fraction of Flux Within SN Radius', fontdict=font_axes)
        axes[i].set_ylabel('Cumulative Fraction of SNe', fontdict=font_axes)
        axes[i].legend(loc='lower right',prop={'size': 16})
        axes[i].set_xlim([0.0,1.0])
        axes[i].set_ylim([0.0,1.0])
        axes[i].text(0.054, 0.85,'%s (%s)' % (bands[i],wavelengths[i]), fontdict=font)  
        axes[i].plot(axes[i].get_xlim(), axes[i].get_ylim(), ls="--", color='black')
        axes[i].tick_params(direction='in',axis='both', which='major', labelsize=12)
        axes[i].grid(linestyle='--',alpha=0.5,zorder=0)
        
    plt.tight_layout()
    plt.savefig('../sne_project/results/%s.pdf' % (res), dpi=300)
    plt.show()
    return(types)


In [ ]:
cdf_2p0 = plot_cdfs(sn_conv2p0, '2kpc')

Plot the CDFs for the SFR tracer.

In [ ]:
def plot_cdfs_sfr(samp, res):
# stuff for the plots
    font = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 18,
            }

    font1 = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'light',
            'style': 'italic',
            'size': 14,
            }
    font_axes = {'family': 'sans-serif',
                'color':  'black',
                'weight': 'normal',
                'size': 16,
                }
    
    legend  = ['Ia', 'II', 'Ib/c']
    colors = ['tab:blue', 'tab:red', 'tab:orange']
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    axes = axes.flatten()
    
    sn_type  = ['Ia', 'II',  'Ib']
    alias1   = ['Ia', 'II P','Ic']
    alias2   = ['Ia', 'IIL', 'Ib/c']
    alias3   = ['Ia', 'II',  'Ibc']
    alias4   = ['Ia', 'II',  'IIb']
    bands    = ['SFR_FUVW4','SFR_NUVW3']
    
    for i in range(len(bands)):
        # plot the toy models
        axes[i].plot(np.array(extended), np.linspace(0,1.0,len(extended)), color='#666666', ls='--', alpha=0.5)
        axes[i].plot(np.array(concentrated), np.linspace(0,1.0,len(concentrated)), color='#666666', ls='--',alpha=0.5)
            

    # for KS test
    Ia  = []
    II  = []
    Ibc = []
    types = [Ia, II, Ibc]

    # loop over each SN type
    gals = [[],[],[],[],[],[]]
    for j in range(len(sn_type)):

        # narrow down the parent table by SN type
        bitmask = (((samp.TYPE == sn_type[j]) |
                    (samp.TYPE == alias1[j]) |
                    (samp.TYPE == alias2[j]) |
                    (samp.TYPE == alias3[j]) |
                    (samp.TYPE == alias4[j])))
        samp_type = samp[bitmask]

        # loop over each SN of a particular type
        cdf_vals = [[],[]]
        sn_rads = []
        bar = ProgressBar(len(samp_type), ipython_widget=True)
        for index, row in samp_type.iterrows():
            pgc = row['PGC']
            sn  = row['NAME']

            # grab the galaxy cdfs
            for i in range(len(bands)):
                try:
                    gal_cdf_file = pd.read_csv('../sne_project/galaxy_control_tables/gal_cdfs_%s/%s_%s_%s.csv' % (res, pgc, bands[i], res))
                except FileNotFoundError:
                    continue
                
                # grab the columns of the galaxy cdfs
                
                rad = np.array(round(gal_cdf_file['RADIUS'],5)) # round so np.interp works
                cdf = np.array(gal_cdf_file['CDF'])

                try:
                    sn_rad = round(row['SN_RAD'],5)
                    sn_rads += [sn_rad]

                    # get SN cdf val     
                    sn_val_cdf = np.interp(sn_rad, rad, cdf)
                    gals[i].append(pgc)
                except:
                    sn_val_cdf = np.nan

                # SN cdf vals
                cdf_vals[i].append(sn_val_cdf)
                
            bar.update()
        # make the plots per band
        for i in range(len(bands)):
            
            # plot the SN distributions
            cdf_vals[i] = np.array(cdf_vals[i])
            cdf_vals[i] = cdf_vals[i][np.isfinite(cdf_vals[i])]
            cdf_vals[i] = np.sort(cdf_vals[i])
            types[j].append(cdf_vals[i]) # FOR KS TEST
            axes[i].plot(cdf_vals[i], np.linspace(0, 1.0, len(cdf_vals[i])), color=colors[j], label=legend[j] + ' (%s SNe)' % (len(cdf_vals[i])), linewidth=2)
   
    # make the plots prettier 
    for i in range(len(bands)):
        axes[i].set_xlabel('Fraction of SFR Within SN Radius', fontdict=font_axes)
        axes[i].set_ylabel('Cumulative Fraction of SNe', fontdict=font_axes)
        axes[i].legend(loc='lower right',prop={'size': 16})
        axes[i].set_xlim([0.0,1.0])
        axes[i].set_ylim([0.0,1.0])
        
        if bands[i] == 'SFR_FUVW4':
            axes[i].text(0.054, 0.85,'$\Sigma_{\mathrm{SFR}_{\mathrm{(FUV+W4)}}}$', fontdict=font, fontsize=22)
        elif bands[i] == 'SFR_NUVW3':
            axes[i].text(0.054, 0.85,'$\Sigma_{\mathrm{SFR}_{\mathrm{(NUV+W3)}}}$', fontdict=font, fontsize=22) 
        axes[i].plot(axes[i].get_xlim(), axes[i].get_ylim(), ls="--", color='black')
        axes[i].tick_params(direction='in',axis='both', which='major', labelsize=12)
        axes[i].grid(linestyle='--',alpha=0.5,zorder=0)
        
    plt.tight_layout()
    plt.savefig('../sne_project/results/%s_SFR_full.pdf' % (res), dpi=300)
    plt.show()
    
    return(types)

In [ ]:
cdf_2p0_sfr = plot_cdfs_sfr(sn_conv2p0, '2kpc')

### Combine the above plots with the MC plots from monte_carlo.ipynb

In [ ]:
# stuff for the plots
font = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 18,
        }

font1 = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'light',
        'style': 'italic',
        'size': 14,
        }
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 16,
            }

colors    = ['tab:blue', 'tab:red', 'tab:orange']
fig, axes = plt.subplots(4, 3, figsize=(15,14.5))
axes = axes.flatten()

sne   = ['Ia', 'II', 'Ibc']
legend = ['SNe Ia', 'SNe II', 'SNe Ib/c']
color = ['tab:blue','tab:red','tab:orange']

# toggle the bands, labels, and wavelengths
bands = ['W1','W2','W3','W4']
label = ['W1','W2','W3','W4']
wavelengths = ['3.4 $\mu$m','4.5 $\mu$m','12 $\mu$m','22 $\mu$m']

result = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')

axis = 0
# loop over each band
for band in range(len(bands)):
    # loop over each SN type
    for tipe in range(len(sne)):
        
        # plot the toy models
        axes[axis].plot(np.array(extended), np.linspace(0,1.0,len(extended)), color='gray', ls='--', alpha=0.7)
        axes[axis].plot(np.array(concentrated), np.linspace(0,1.0,len(concentrated)), color='gray', ls='--',alpha=0.7)

        # grab the monte carlo distributions and the real SN distributions
        distributions = pd.read_csv('../sne_project/results/monte_carlo/%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        sne_dist = result['%s_%s' % (bands[band],sne[tipe])].dropna()
            
        # plot the monte carlo distributions
        for i in range(0,100):
            dist = distributions['DIST_%s' % (i)]
            axes[axis].plot(dist,np.linspace(0, 1.0, len(dist)), color='gray',alpha=0.25)
            
        # plot the real SN distribution
        axes[axis].plot(sne_dist,np.linspace(0, 1.0, len(sne_dist)),color=color[tipe],
                        label=legend[tipe] + ' (%s SNe)' % (len(sne_dist)),linewidth=4,zorder=10)
        axes[axis].set_xlim([0.0,1.0])
        axes[axis].set_ylim([-0.005,1.0])
        
        # add labels for the SN type and band
        axes[axis].text(0.054, 0.85,'%s (%s)' % (bands[band],wavelengths[band]), fontdict=font) 
        axes[axis].plot(axes[axis].get_xlim(), axes[axis].get_ylim(), ls='--', color='black')
        axes[axis].tick_params(direction='in',axis='both', which='major', labelsize=13, length=5)
        axes[axis].grid(linestyle='--',alpha=0.5,zorder=0)
        axes[axis].legend(loc='lower right',prop={'size': 15})

        if (bands[band] == 'W4'):
            axes[axis].set_xlabel('Fraction of Flux Within SN Radius', fontdict=font_axes)
        if (sne[tipe] == 'Ia'):
            axes[axis].set_ylabel('Cumulative Fraction of SNe',  fontsize=16)
    
        axis = axis + 1
        
plt.tight_layout()
plt.savefig('../sne_project/results/final_results_IR.pdf', dpi=300)
plt.show()


In [ ]:
# stuff for the plots
font = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 18,
        }

font1 = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'light',
        'style': 'italic',
        'size': 14,
        }
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 16,
            }

colors    = ['tab:blue', 'tab:red', 'tab:orange']
fig, axes = plt.subplots(4, 3, figsize=(15,14.5))
axes = axes.flatten()

sne   = ['Ia', 'II', 'Ibc']
legend = ['SNe Ia', 'SNe II', 'SNe Ib/c']
color = ['tab:blue','tab:red','tab:orange']

# toggle the labels, bands, and wavelengths
bands = ['NUV','FUV','SFR_NUVW3','SFR_FUVW4']
label = ['NUV','FUV','$\Sigma_{\mathrm{SFR}_{\mathrm{NUV+W3}}}$','$\Sigma_{\mathrm{SFR}_{\mathrm{FUV+W4}}}$']
wavelengths = ['231 nm','154 nm']

result_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
result     = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')


axis = 0
# loop over each band
for band in range(len(bands)):
    # loop over each SN type
    for tipe in range(len(sne)):
        
        # plot the toy models
        axes[axis].plot(np.array(extended), np.linspace(0,1.0,len(extended)), color='gray', ls='--', alpha=0.7)
        axes[axis].plot(np.array(concentrated), np.linspace(0,1.0,len(concentrated)), color='gray', ls='--',alpha=0.7)
        
        # grab the monte carlo distributions and the real SN distributions
        distributions = pd.read_csv('../sne_project/results/monte_carlo/%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
            sne_dist = result_sfr['%s_%s' % (bands[band],sne[tipe])].dropna()
        else:
            sne_dist = result['%s_%s' % (bands[band],sne[tipe])].dropna()
    
        # plot the monte carlo distributions
        for i in range(0,100):
            dist = distributions['DIST_%s' % (i)]
            axes[axis].plot(dist,np.linspace(0, 1.0, len(dist)), color='gray',alpha=0.25)
            
        # plot the real SN distribution
        axes[axis].plot(sne_dist,np.linspace(0, 1.0, len(sne_dist)),color=color[tipe],
                        label=legend[tipe] + ' (%s SNe)' % (len(sne_dist)),linewidth=4,zorder=10)
        axes[axis].set_xlim([0.0,1.0])
        axes[axis].set_ylim([-0.005,1.0])
        
        # add labels for the SN type and band
        
        if (label[band] == 'NUV') or (label[band] == 'FUV'):
            axes[axis].text(0.054, 0.85,'%s (%s)' % (label[band],wavelengths[band]), fontdict=font)
        else:
            axes[axis].text(0.054, 0.85,'%s' % (label[band]), fontdict=font, fontsize=22)
        axes[axis].plot(axes[axis].get_xlim(), axes[axis].get_ylim(), ls='--', color='black')
        axes[axis].tick_params(direction='in',axis='both', which='major', labelsize=13, length=5)
        axes[axis].grid(linestyle='--',alpha=0.5,zorder=0)
        axes[axis].legend(loc='lower right',prop={'size': 15})
        
        if (bands[band] == 'FUV'):
            axes[axis].set_xlabel('Fraction of Flux Within SN Radius', fontdict=font_axes)
        if (bands[band] == 'SFR_FUVW4'):
            axes[axis].set_xlabel('Fraction of SFR Within SN Radius', fontdict=font_axes)
        if sne[tipe] == 'Ia':
            axes[axis].set_ylabel('Cumulative Fraction of SNe',  fontsize=16)
    
        axis = axis + 1
        
plt.tight_layout()
plt.savefig('../sne_project/results/final_results_UVSFR.pdf', dpi=300)
plt.show()

### Output the CDFs to file so that we can run statistical tests

Output the CDFs of the regular bands.

In [ ]:
def output_final_cdfs(band_cdfs_per_type, res):

                                         # type # band
    w1_ia  = pd.DataFrame(band_cdfs_per_type[0][0], columns=['W1_Ia'])
    w1_ii  = pd.DataFrame(band_cdfs_per_type[1][0], columns=['W1_II'])
    w1_ibc = pd.DataFrame(band_cdfs_per_type[2][0], columns=['W1_Ibc'])

    w2_ia  = pd.DataFrame(band_cdfs_per_type[0][1], columns=['W2_Ia'])
    w2_ii  = pd.DataFrame(band_cdfs_per_type[1][1], columns=['W2_II'])
    w2_ibc = pd.DataFrame(band_cdfs_per_type[2][1], columns=['W2_Ibc'])

    w3_ia  = pd.DataFrame(band_cdfs_per_type[0][2], columns=['W3_Ia'])
    w3_ii  = pd.DataFrame(band_cdfs_per_type[1][2], columns=['W3_II'])
    w3_ibc = pd.DataFrame(band_cdfs_per_type[2][2], columns=['W3_Ibc'])

    w4_ia  = pd.DataFrame(band_cdfs_per_type[0][3], columns=['W4_Ia'])
    w4_ii  = pd.DataFrame(band_cdfs_per_type[1][3], columns=['W4_II'])
    w4_ibc = pd.DataFrame(band_cdfs_per_type[2][3], columns=['W4_Ibc'])

    nuv_ia  = pd.DataFrame(band_cdfs_per_type[0][4], columns=['NUV_Ia'])
    nuv_ii  = pd.DataFrame(band_cdfs_per_type[1][4], columns=['NUV_II'])
    nuv_ibc = pd.DataFrame(band_cdfs_per_type[2][4], columns=['NUV_Ibc'])

    fuv_ia  = pd.DataFrame(band_cdfs_per_type[0][5], columns=['FUV_Ia'])
    fuv_ii  = pd.DataFrame(band_cdfs_per_type[1][5], columns=['FUV_II'])
    fuv_ibc = pd.DataFrame(band_cdfs_per_type[2][5], columns=['FUV_Ibc'])

    total = pd.concat([w1_ia,w1_ii,w1_ibc,w2_ia,w2_ii,w2_ibc,w3_ia,w3_ii,w3_ibc,w4_ia,w4_ii,w4_ibc,
                      nuv_ia,nuv_ii,nuv_ibc,fuv_ia,fuv_ii,fuv_ibc], axis=1)
    
    total.to_csv('../sne_project/results/ks_test/%s.csv' % (res), index=False)
    return

In [ ]:
output_final_cdfs(cdf_2p0, '2kpc')

Now output the CDFs for the SFR tracer.

In [ ]:
def output_final_sfr_cdf(cdfs_per_type, res):
    
    fuvw4_ia  = pd.DataFrame(cdfs_per_type[0][0], columns=['SFR_FUVW4_Ia'])
    fuvw4_ii  = pd.DataFrame(cdfs_per_type[1][0], columns=['SFR_FUVW4_II'])
    fuvw4_ibc = pd.DataFrame(cdfs_per_type[2][0], columns=['SFR_FUVW4_Ibc'])
    
    nuvw3_ia  = pd.DataFrame(cdfs_per_type[0][1], columns=['SFR_NUVW3_Ia'])
    nuvw3_ii  = pd.DataFrame(cdfs_per_type[1][1], columns=['SFR_NUVW3_II'])
    nuvw3_ibc = pd.DataFrame(cdfs_per_type[2][1], columns=['SFR_NUVW3_Ibc'])

    total = pd.concat([fuvw4_ia,fuvw4_ii,fuvw4_ibc,nuvw3_ia,nuvw3_ii,nuvw3_ibc], axis=1)

    total.to_csv('../sne_project/results/ks_test/SFR_%s.csv' % (res), index=False)
    
    return

In [ ]:
output_final_sfr_cdf(cdf_2p0_sfr, '2kpc')

# STATS STATS STATS STATS

### KS Tests

Run against host galaxy emission.

Define a function that will use the statistic number (D) from the KS-test to tell us if the null hypothesis can be rejected.

In [ ]:
def reject(m, n, D):
    calpha = 1.358
    
    val = calpha * np.sqrt((m+n) / (m*n))
    
    if D > val:
        output = 'reject'
    else:
        output = 'cannot reject'
    return(output, val)

### KS test of each gray MC distribution against the SN distribution.

In [ ]:
bar = ProgressBar(range(len(bands)), ipython_widget=True)
for band in range(len(bands)):
    bar.update()
    # loop over each SN type
    for tipe in range(len(sne)):
        # grab the monte carlo distributions
        distributions = pd.read_csv('../sne_project/results/monte_carlo/%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        
        # grab the cdfs
        if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
            cdfs_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
            cdf_sfr  = cdfs_sfr['%s_%s' % (bands[band],sne[tipe])].dropna()
        else:
            cdfs = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')
            cdf  = cdfs['%s_%s' % (bands[band],sne[tipe])].dropna()

        stats = []
        pvals = []
        dists = []
        ks_df = pd.DataFrame()
        for i in range(0, 100):
            dist = distributions['DIST_%s' % (i)]
            
            if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
                stat,pval = ks_2samp(dist, cdf_sfr)
            else:
                stat,pval = ks_2samp(dist, cdf)
            
            stats.append(stat)
            pvals.append(pval)
            dists.append('DIST_%s' % (i))
            
        ks_df['DIST'] = dists
        ks_df['STATISTIC'] = stats
        ks_df['PVALUE'] = pvals
            
        ks_df.to_csv('../sne_project/results/monte_carlo/gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]), index=False)

Make a table of these KS test results.

In [ ]:
def ks_test_mc(cdfs, ks_test_df, sfr):
    which_cdf = []
    statistic = []
    pvalues   = []
    thresholds = []
    results   = []
    
    if sfr == True:
        bands = ['SFR_FUVW4','SFR_NUVW3']
    else:
        bands = ['W1','W2','W3','W4','NUV','FUV']
    for band in range(len(bands)):
        # loop over each SN type
        for tipe in range(len(sne)):
            ks = pd.read_csv('../sne_project/results/monte_carlo/gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
            
            sn_cdf = cdfs['%s_%s' % (bands[band],sne[tipe])].dropna()

            # get the median of the pvalues
            pvals = np.array(ks['PVALUE']) 
            pval = np.median(pvals)

            # get the median of the statistic D values
            dvals = np.array(ks['STATISTIC']) 
            dval = np.median(dvals)

            result, thresh = reject(len(sn_cdf), len(sn_cdf), dval)
            
            which_cdf.append('%s_%s' % (bands[band],sne[tipe]))
            pvalues.append(pval)
            statistic.append(dval)
            thresholds.append(thresh)
            results.append(result)
    
    ks_test_df['CDF']       = which_cdf
    ks_test_df['STATISTIC'] = statistic
    ks_test_df['PVALUE']    = pvalues
    ks_test_df['THRESHOLD'] = thresholds
    ks_test_df['REJECT']    = results
    return(ks_test_df)

In [ ]:
ks_test = pd.DataFrame()
cdfs_2kpc = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')
ks_test_2kpc = ks_test_mc(cdfs_2kpc, ks_test, False)

ks_test = pd.DataFrame()
cdfs_2kpc_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
ks_test_2kpc_sfr = ks_test_mc(cdfs_2kpc_sfr, ks_test, True)

In [ ]:
ks_test_2kpc = ks_test_2kpc.append(ks_test_2kpc_sfr)
ks_test_2kpc.to_csv('../sne_project/results/ks_test/ks_2kpc_mc.csv', index=False)
ks_test_2kpc

### Get the uncertainties of the SN p-values

Use the p-values obtained from doing the KS test between the SN distributions and each gray curve. Take the median of these p-values to represent the SN distribution vs. the host galaxy emission.

In [ ]:
bar = ProgressBar(range(len(bands)), ipython_widget=True)
for band in range(len(bands)):
    bar.update()
    print(bands[band])
    # loop over each SN type
    for tipe in range(len(sne)):
        print(sne[tipe])
        ks = pd.read_csv('../sne_project/results/monte_carlo/gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        pval = np.array(ks['PVALUE'])
        pval = np.sort(pval)
        
        percent16 = np.percentile(pval, 16)
        percent84 = np.percentile(pval, 84)
        
        # using the p-values from averaging the MC distributions and testing 
        sn_ks_full = pd.read_csv('../sne_project/results/ks_test/ks_2kpc_mc.csv')
        sn_ks = sn_ks_full[sn_ks_full['CDF'] == '%s_%s' % (bands[band],sne[tipe])]
        sn_pval = float(sn_ks['PVALUE'])
        
        print('+',percent84 - sn_pval)
        print('-', sn_pval - percent16)

### Get the uncertainties of the SN D values

Use the D values obtained from doing the KS test between the SN distributions and each gray curve. Take the median of these D values to represent the SN distribution vs. the host galaxy emission.

In [ ]:
bands = ['W1','W2','W3','W4','NUV','FUV','SFR_FUVW4','SFR_NUVW3']
sne   = ['Ia','II','Ibc']
bar = ProgressBar(range(len(bands)), ipython_widget=True)
for band in range(len(bands)):
    bar.update()
    print(bands[band])
    # loop over each SN type
    for tipe in range(len(sne)):
        print(sne[tipe])
        ks = pd.read_csv('../sne_project/results/monte_carlo/gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        dval = np.array(ks['STATISTIC'])
        dval = np.sort(dval)
        
        percent16 = np.percentile(dval, 16)
        percent84 = np.percentile(dval, 84)
        
        # using the p-values from averaging the MC distributions and testing 
        sn_ks_full = pd.read_csv('../sne_project/results/ks_test/ks_2kpc_mc.csv')
        sn_ks = sn_ks_full[sn_ks_full['CDF'] == '%s_%s' % (bands[band],sne[tipe])]
        sn_dval = float(sn_ks['STATISTIC'])
        
        print('+',percent84 - sn_dval)
        print('-',sn_dval - percent16)

## Do the above but with the Anderson-Darling test!

### AD test of each gray distribution against the SN distribution.

In [ ]:
bar = ProgressBar(range(len(bands)), ipython_widget=True)
for band in range(len(bands)):
    bar.update()
    # loop over each SN type
    for tipe in range(len(sne)):
        # grab the monte carlo distributions
        distributions = pd.read_csv('../sne_project/results/monte_carlo/%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        
        # grab the cdfs
        if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
            cdfs_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
            cdf_sfr  = cdfs_sfr['%s_%s' % (bands[band],sne[tipe])].dropna()
        else:
            cdfs = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')
            cdf  = cdfs['%s_%s' % (bands[band],sne[tipe])].dropna()

        stats  = []
        cvals  = []
        dists  = []
        levels = []
        ad_df  = pd.DataFrame()
        for i in range(0, 100):
            dist = distributions['DIST_%s' % (i)]
            
            if (bands[band] == 'SFR_FUVW4') or (bands[band] == 'SFR_NUVW3'):
                try:
                    stat,cval,level = anderson_ksamp([dist, cdf_sfr])
                    stats.append(stat)
                    cvals.append(cval)
                    levels.append(level)
                except:
                    stats.append(np.nan)
                    cvals.append(np.nan)
                    levels.append(np.nan)
            else:
                try:
                    stat,cval,level = anderson_ksamp([dist, cdf])
                    stats.append(stat)
                    cvals.append(cval)
                    levels.append(level)
                except:
                    stats.append(np.nan)
                    cvals.append(np.nan)
                    levels.append(np.nan)
        
            dists.append('DIST_%s' % (i))
            
        ad_df['DIST'] = dists
        ad_df['STATISTIC'] = stats
        ad_df['CRITICAL VALUES'] = cvals
        ad_df['SIGNIFICANCE LEVEL'] = levels
            
        ad_df.to_csv('../sne_project/results/monte_carlo/ad_gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]), index=False)

Make a table of these AD test results.

In [ ]:
def ad_test_mc(cdfs, ad_test_df, sfr):
    which_cdf = []
    statistics = []
    cvalues   = []
    levels = []
    rejection = []
    
    if sfr == True:
        bands = ['SFR_FUVW4','SFR_NUVW3']
    else:
        bands = ['W1','W2','W3','W4','NUV','FUV']
    for band in range(len(bands)):
        # loop over each SN type
        for tipe in range(len(sne)):
            ad = pd.read_csv('../sne_project/results/monte_carlo/ad_gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
            
            sn_cdf = cdfs['%s_%s' % (bands[band],sne[tipe])].dropna()

            # get the median of the statistic value
            stats = np.array(ad['STATISTIC']) 
            stat = np.median(stats)

            # get the median of the critical values
            cvals = ad['CRITICAL VALUES'].iloc[0]
            
            # get the sig levels
            level = ad['SIGNIFICANCE LEVEL']
            level = np.median(level)
            
            if level < 0.05:
                rejection.append('REJECT')
            else:
                rejection.append('CANNOT REJECT')
            
            which_cdf.append('%s_%s' % (bands[band],sne[tipe]))
            statistics.append(stat)
            cvalues.append(cvals)
            levels.append(level)

    ad_test_df['CDF']       = which_cdf
    ad_test_df['STATISTIC'] = statistics
    ad_test_df['CRITICAL VALUES'] = cvalues
    ad_test_df['SIGNIFICANCE LEVEL'] = levels
    ad_test_df['REJECTION'] = rejection
    return(ad_test_df)


In [ ]:
ad_test = pd.DataFrame()
cdfs_2kpc = pd.read_csv('../sne_project/results/ks_test/2kpc.csv')
ad_test_2kpc = ad_test_mc(cdfs_2kpc, ad_test, False)

ad_test = pd.DataFrame()
cdfs_2kpc_sfr = pd.read_csv('../sne_project/results/ks_test/SFR_2kpc.csv')
ad_test_2kpc_sfr = ad_test_mc(cdfs_2kpc_sfr, ad_test, True)

In [ ]:
ad_test_2kpc = ad_test_2kpc.append(ad_test_2kpc_sfr)
ad_test_2kpc.to_csv('../sne_project/results/ks_test/ad_2kpc_mc.csv', index=False)
ad_test_2kpc

### Get the uncertainties of the SN statistic values (AD test)

Use the statistic values obtained from doing the AD test between the SN distributions and each gray curve. Take the median of these statistic values to represent the SN distribution vs. the host galaxy emission.

In [ ]:
bands = ['W1','W2','W3','W4','NUV','FUV','SFR_FUVW4','SFR_NUVW3']
sne   = ['Ia','II','Ibc']
bar = ProgressBar(range(len(bands)), ipython_widget=True)
for band in range(len(bands)):
    bar.update()
    print(bands[band])
    # loop over each SN type
    for tipe in range(len(sne)):
        print(sne[tipe])
        ad = pd.read_csv('../sne_project/results/monte_carlo/ad_gray_sn_%s_%s_2kpc.csv' % (bands[band],sne[tipe]))
        stats = np.array(ad['STATISTIC'])
        stats = np.sort(stats)
        
        percent16 = np.percentile(stats, 16)
        percent84 = np.percentile(stats, 84)
        
        # using the p-values from averaging the MC distributions and testing 
        sn_ad_full = pd.read_csv('../sne_project/results/ks_test/ad_2kpc_mc.csv')
        sn_ad = sn_ad_full[sn_ad_full['CDF'] == '%s_%s' % (bands[band],sne[tipe])]
        
        sn_stat = float(sn_ad['STATISTIC'])
        
        print('+',percent84 - sn_stat)
        print('-',sn_stat - percent16)

### Find the percentage of D-values greater than D_SN.

- D_SN = the D-value from the KS test between the SN distributions and the 1:1 line. 
- D = the D-values from the KS test between the MC distributions and the 1:1 line.

In [ ]:
types = ['Ia','II','Ibc']
bands = ['W1','W2','W3','W4','NUV','FUV', 'SFR_FUVW4','SFR_NUVW3']
ks_diag = pd.read_csv('../sne_project/results/ks_test/ks_2kpc.csv')
for band in bands:
    for tipe in types:
        
        # grab the MC vs. 1:1 line results
        results_mc = pd.read_csv('../sne_project/results/monte_carlo/gray_%s_%s_2kpc.csv' % (band,tipe))
        dval_mc = np.array(results_mc['STATISTIC'])
        
        # grab the SN vs. 1:1 line results
        results_sn = ks_diag[ks_diag['CDF'] == '%s_%s' % (band,tipe)]
        dval_sn = float(results_sn['STATISTIC (10000)'])
        
        print('%s_%s: '% (band,tipe), len(np.where(dval_mc > dval_sn)[0])/100.)

# - Extra plots for the paper -

### Example host galaxy W1 images (with SNe marked).

In [ ]:
sn_conv2p0 = pd.read_csv('sample_processing/2kpc.csv')
gal = sn_conv2p0[sn_conv2p0['PGC'] == 'PGC13179']

In [ ]:
plt.subplots_adjust(wspace=0,hspace=0)
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

#################################################################

sample = pd.read_csv('samples/sne_sample.csv')
gal    = sample[sample['PGC'] == 'PGC13179']
ras   = gal['RA']
decs  = gal['DEC']
names = gal['NAME']
types = gal['TYPE']

hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/delivery/PGC13179_w1_gauss7p5.fits')
mapp = hdulist[0].data
w = WCS(hdulist[0].header)
newmap = mapp.copy()

# using vmin and vmax from the paper (logarithmic stretch)
newmap[newmap < 10**-2] = 10**-2
newmap[newmap > 10**1.5] = 10**1.5

# take log of map
logmap = np.log10(newmap)

coords_arr = np.column_stack((ras, decs))
coords = w.wcs_world2pix(coords_arr, 0)
x = coords[:,0]
y = coords[:,1]

# show the image with sne
ax1.imshow(logmap, origin='lower', interpolation='nearest', cmap=cm.gray_r, zorder=1)
ax1.plot(x[0], y[0], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax1.plot(x[1], y[1], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax1.plot(x[2], y[2], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax1.text(x[0]+70, y[0]+10, 'SN2012fr (Ia)', color='black', fontsize=15, family='sans-serif') #F49D37ax
ax1.text(x[1]+68, y[1]-10, 'SN2001du (II)', color='black', fontsize=15, family='sans-serif') #F49D37
ax1.text(x[2]+60, y[2]-45, 'SN1983V (Ic)', color='black', fontsize=15, family='sans-serif')

ax1.text(0.2, 0.85, 'NGC 1365', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax1.transAxes)
ax1.text(0.2, 0.8, 'PGC 13179', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax1.transAxes)

ax1.axis('off')
ax1.set_xticklabels([])
ax1.set_yticklabels([])

#################################################################

sample = pd.read_csv('samples/sne_sample.csv')
gal    = sample[sample['PGC'] == 'PGC42064']
ras   = gal['RA']
decs  = gal['DEC']
names = gal['NAME']
types = gal['TYPE']

hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/delivery/PGC42064_w1_gauss7p5.fits')
mapp = hdulist[0].data
w = WCS(hdulist[0].header)
newmap = mapp.copy()

# using vmin and vmax from the paper (logarithmic stretch)
newmap[newmap < 10**-2] = 10**-2
newmap[newmap > 10**1.5] = 10**1.5

# take log of map
logmap = np.log10(newmap)

coords_arr = np.column_stack((ras, decs))
coords = w.wcs_world2pix(coords_arr, 0)
x = coords[:,0]
y = coords[:,1]

# show the image with sne
ax2.imshow(logmap, origin='lower', interpolation='nearest', cmap=cm.gray_r, zorder=1)
ax2.plot(x[0], y[0], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax2.plot(x[1], y[1], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax2.text(x[0]+10, y[0]-5, 'SN1990B (Ic)', color='black', fontsize=15, family='sans-serif')
ax2.text(x[0]-85, y[0], 'SN2004cc (Ib)', color='black', fontsize=15, family='sans-serif')

ax2.text(0.2, 0.85, 'NGC 4568', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax2.transAxes)
ax2.text(0.2, 0.8, 'PGC 42064', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax2.transAxes)

ax2.axis('off')
ax2.set_xticklabels([])
ax2.set_yticklabels([])

#################################################################

sample = pd.read_csv('samples/sne_sample.csv')
gal    = sample[sample['PGC'] == 'PGC40455']
ras   = gal['RA']
decs  = gal['DEC']
names = gal['NAME']
types = gal['TYPE']

hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/delivery/PGC40455_w1_gauss7p5.fits')
mapp = hdulist[0].data
w = WCS(hdulist[0].header)
newmap = mapp.copy()

# using vmin and vmax from the paper (logarithmic stretch)
newmap[newmap < 10**-2] = 10**-2
newmap[newmap > 10**1.5] = 10**1.5

# take log of map
logmap = np.log10(newmap)

coords_arr = np.column_stack((ras, decs))
coords = w.wcs_world2pix(coords_arr, 0)
x = coords[:,0]
y = coords[:,1]

# show the image with sne
ax3.imshow(logmap, origin='lower', interpolation='nearest', cmap=cm.gray_r, zorder=1)
ax3.plot(x[0], y[0], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax3.text(x[0]+45, y[0]+3, 'SN1957B (Ia)', color='black', fontsize=15, family='sans-serif')
ax3.text(0.2, 0.85, 'NGC 4374', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax3.transAxes)
ax3.text(0.2, 0.8, 'PGC 40455', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax3.transAxes)
ax3.axis('off')
ax3.set_xticklabels([])
ax3.set_yticklabels([])

#################################################################

sample = pd.read_csv('samples/sne_sample.csv')
gal    = sample[sample['PGC'] == 'PGC68941']
ras   = gal['RA']
decs  = gal['DEC']
names = gal['NAME']
types = gal['TYPE']

hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/delivery/PGC68941_w1_gauss7p5.fits')
mapp = hdulist[0].data
w = WCS(hdulist[0].header)
newmap = mapp.copy()

# using vmin and vmax from the paper (logarithmic stretch)
newmap[newmap < 10**-2] = 10**-2
newmap[newmap > 10**1.5] = 10**1.5

# take log of map
logmap = np.log10(newmap)

coords_arr = np.column_stack((ras, decs))
coords = w.wcs_world2pix(coords_arr, 0)
x = coords[:,0]
y = coords[:,1]

# show the image with sne
ax4.imshow(logmap, origin='lower', interpolation='nearest', cmap=cm.gray_r, zorder=1)
ax4.plot(x[0], y[0], 'o', fillstyle='none', ms=7, color='w', zorder=10, mew=2)
ax4.text(x[0]+10, y[0], 'SN1964H (II)', color='black', fontsize=15, family='sans-serif')
ax4.text(0.2, 0.85, 'NGC 7292', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax4.transAxes)
ax4.text(0.2, 0.8, 'PGC 68941', color='black', fontsize=16, verticalalignment='center', horizontalalignment='center',
         family='sans-serif',fontweight='bold',transform=ax4.transAxes)
ax4.axis('off')
ax4.set_xticklabels([])
ax4.set_yticklabels([])

plt.axis('off')
# plt.tight_layout()
plt.savefig('results/fig1.png', dpi=300)
plt.show()
plt.close()

### Example host galaxy CDF (with SNe marked).

In [ ]:
gal_cdf = pd.read_csv('galaxy_control_tables/gal_cdfs_2kpc/PGC13179_W1_2kpc.csv')
gal = sn_conv2p0[sn_conv2p0['PGC'] == 'PGC13179']
R25 = gal['R25'].iloc[0]

# round radius values so that np.interp works
rad = np.array(round(gal_cdf['RADIUS'],5))
cdf = np.array(gal_cdf['CDF'])
  
# get SN radii
sn_rad = round(gal['SN_RAD'],5)

# get SN cdf val     
sn_val_cdf = np.interp(sn_rad, rad, cdf)

In [ ]:
# stuff for the plots
font = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 18,
        }

font1 = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 16,
            }

fig = plt.figure(figsize=(6,4))
colors = ['tab:blue', 'tab:red', 'tab:green']

# make x-axis in terms of [R25]
plt.plot(rad/(R25), cdf, color='#666666', lw=3)
plt.xlim([0,2])
plt.ylim([0,1])
plt.xlabel('Galactocentric Radius [$r_{25}$]', fontdict=font_axes)
plt.ylabel('Cumulative Frac. of Flux', fontdict=font_axes)
plt.tick_params(direction='in',axis='both', which='major', labelsize=12)
plt.text(0.15, 0.85,'W1', fontdict=font) 
plt.grid(linestyle='--',alpha=0.5,zorder=0)

plt.plot(np.array(sn_rad)[0]/(R25), sn_val_cdf[0], '*', color='#2bace5', mew=1.5, ms='20',mec='black') # SN2012fr (Ia)
plt.plot(np.array(sn_rad)[1]/(R25), sn_val_cdf[1], '*', color='red', mew=1.5,ms='20',mec='black') 
plt.plot(np.array(sn_rad)[2]/(R25), sn_val_cdf[2], '*', color='tab:orange',mew=1.5, ms='20',mec='black') # SN1983V (Ic)

plt.text((np.array(sn_rad)[0])/(R25) + 0.05, np.array(sn_val_cdf)[0] - 0.075, 'SN2012fr (Ia)', fontdict=font_axes)
plt.text((np.array(sn_rad)[1])/(R25) + 0.05, np.array(sn_val_cdf)[1] - 0.05, 'SN2001du (II)', fontdict=font_axes)
plt.text((np.array(sn_rad)[2])/(R25) + 0.05, np.array(sn_val_cdf)[2] - 0.05, 'SN1983V (Ic)', fontdict=font_axes)

# plt.vlines((np.array(sn_rad)[0])/(2*R25), 0, np.array(sn_val_cdf)[0], color='#BB0000')
# plt.vlines((np.array(sn_rad)[1])/(2*R25), 0, np.array(sn_val_cdf)[1], color='#BB0000')
# plt.vlines((np.array(sn_rad)[2])/(2*R25), 0, np.array(sn_val_cdf)[2], color='#BB0000')
# plt.text(np.array(sn_rad)+0.1, np.array(sn_val_cdf)+0.1, my_sne, color='#BB0000')

plt.tight_layout()
plt.savefig('results/gal_cdf.pdf', dpi=300)
plt.show()

### Intermediate step to show compactness of SN distributions

In [ ]:
def cdf_model(l_fid):
    dr = 0.01
    rgal = np.arange(0., 2.0, 0.001)
    fid_disk = np.exp(-1.0*rgal/l_fid)
    annulus_area = 2 * np.pi * rgal * dr
    total_flux_in_annulus = fid_disk * annulus_area
    cdf = np.cumsum(total_flux_in_annulus)
    norm_cdf = cdf / cdf[-1]
    return(fid_disk,norm_cdf)

In [ ]:
# calculate for our galaxy with l_fid = 0.25
fid_disk_stan, cdf_gal = cdf_model(l_fid=0.25)

# calculate for the SN distributions
fid_disk_con, cdf_con = cdf_model(l_fid=0.125) # concentrated
fid_disk_ext, cdf_ext = cdf_model(l_fid=0.5)    # extended

In [ ]:
standard = []
standard_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_stan)/np.sum(2*np.pi*rgal*fid_disk_stan))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    standard.append(random_cdf)
    standard_rad.append(random_rad)

concentrated = []
concentrated_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_con)/np.sum(2*np.pi*rgal*fid_disk_con))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    concentrated.append(random_cdf)
    concentrated_rad.append(random_rad)
    
extended = []
extended_rad = []
for i in range(0, 10000):
    rgal = np.arange(0., 2.0, 0.001)
    random_rad = np.random.choice(a=rgal, p=(2*np.pi*rgal*fid_disk_ext)/np.sum(2*np.pi*rgal*fid_disk_ext))
    random_cdf = np.interp(random_rad, rgal, cdf_gal)
    extended.append(random_cdf)
    extended_rad.append(random_rad)

In [ ]:
standard = np.array(standard)
standard = standard[np.isfinite(standard)]
standard = np.sort(standard)

extended = np.array(extended)
extended = extended[np.isfinite(extended)]
extended = np.sort(extended)

concentrated = np.array(concentrated)
concentrated = concentrated[np.isfinite(concentrated)]
concentrated = np.sort(concentrated)

fig = plt.figure(figsize=(6,4))
font_axes = {'family': 'sans-serif',
            'color':  'black',
            'weight': 'normal',
            'size': 16,
            }

plt.plot(np.array(standard), np.linspace(0,1.0,len(standard)), linestyle='--',color='#666666',lw=3, label='perfect match')
plt.plot(np.array(extended), np.linspace(0,1.0,len(extended)), color='tab:blue',lw=3, label='extended SN distrib.')
plt.plot(np.array(concentrated), np.linspace(0,1.0,len(concentrated)), color='tab:red',lw=3, label='compact SN distrib.')

# plot an arbitrary point
plt.plot(0.6, np.interp(0.6,np.array(standard),np.linspace(0,1.0,len(standard))),marker='*', ms=20,color='gold', mec='black')
plt.plot(0.8, np.interp(0.8,np.array(extended),np.linspace(0,1.0,len(extended))),marker='*', ms=20,color='gold',  mec='black')
plt.plot(0.4, np.interp(0.4,np.array(concentrated),np.linspace(0,1.0,len(concentrated))),marker='*', ms=20,color='gold',  mec='black')
print(np.interp(0.6,np.array(standard),np.linspace(0,1.0,len(standard))))
print(np.interp(0.8,np.array(standard),np.linspace(0,1.0,len(standard))))
print(np.interp(0.4,np.array(standard),np.linspace(0,1.0,len(standard))))


plt.xlim([0.,1.])
plt.ylim([0.,1.])
plt.xlabel('Fraction of Fiducial Flux', fontdict=font_axes)
plt.ylabel('Cumulative Frac. of Sampling', fontdict=font_axes, fontsize=15)
plt.tick_params(direction='in',axis='both', which='major', labelsize=12)
plt.grid(linestyle='--',alpha=0.5,zorder=0)

gray = mlines.Line2D([], [], linestyle='--', lw=3, color='#666666',  label='$\ell = 0.25\ r_{25}$')
blue = mlines.Line2D([], [], linestyle='-', lw=3, color='tab:blue', label='$\ell = 0.5\ r_{25}$')
red  = mlines.Line2D([], [], linestyle='-', lw=3, color='tab:red',  label='$\ell = 0.125\ r_{25}$')
legend2 = plt.legend(handles=[gray,blue,red],loc='upper left',fontsize=11.3)

# add both legends
plt.legend(fontsize=11.25, loc='lower right')
plt.gca().add_artist(legend2)
plt.tight_layout()
# plt.savefig('results/fiducial_flux.pdf',dpi=300)

In [ ]:
colors = ['#666666','tab:blue','tab:red']
legend = ['galaxy model','extended SN distribution','compact SN distribution']
dr = 0.001
rgal = np.arange(0., 2.0, dr)
fig = plt.figure(figsize=(6,4))
l_fids = [0.25,0.5,0.125]
points = [0.6,0.8,0.4]
linestyles = ['--','-','-']
for l_fid in l_fids:
    fid_disk = np.exp(-1.0*rgal/l_fid)
    annulus_area = 2 * np.pi * rgal * dr
    total_flux_in_annulus = fid_disk * annulus_area
    cdf = np.cumsum(total_flux_in_annulus)
    norm_cdf = cdf / cdf[-1]
    plt.plot(rgal, norm_cdf, color=colors[l_fids.index(l_fid)],linestyle=linestyles[l_fids.index(l_fid)],label=legend[l_fids.index(l_fid)],lw=3)
    
    # plot an arbitrary point
    plt.plot(np.interp(points[l_fids.index(l_fid)],norm_cdf,rgal), points[l_fids.index(l_fid)],marker='*', ms=20,color='gold', mec='black')
    print(np.interp(points[l_fids.index(l_fid)],norm_cdf,rgal))

plt.xlim([0.,2.])
plt.ylim([0.,1.])
plt.xlabel('Galactocentric Radius [$r_{25}$]', fontdict=font_axes)
plt.ylabel('Cumulative Frac. of Flux', fontdict=font_axes)
plt.tick_params(direction='in',axis='both', which='major', labelsize=12)
plt.grid(linestyle='--',alpha=0.5,zorder=0)

gray = mlines.Line2D([], [], linestyle='--', lw=3, color='#666666',  label='$\ell = 0.25\ r_{25}$')
blue = mlines.Line2D([], [], linestyle='-', lw=3, color='tab:blue', label='$\ell = 0.5\ r_{25}$')
red  = mlines.Line2D([], [], linestyle='-', lw=3, color='tab:red',  label='$\ell = 0.125\ r_{25}$')
legend2 = plt.legend(handles=[gray,blue,red],loc='center right',fontsize=14)

plt.legend(fontsize=13, loc='lower right')
plt.gca().add_artist(legend2)
plt.tight_layout()
plt.savefig('results/fiducial_gal_cdf.pdf',dpi=300)
plt.show()